Downsample the lung data for comparison to TMS

In [1]:
import anndata as ad
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib as plt

In [2]:
adata = sc.read_h5ad('/home/hkaufm49/working_repo/PanSci/data/GSE247719_PanSci_Myeloid_cell_adata.h5ad')
adata


AnnData object with n_obs × n_vars = 1375734 × 55416
    obs: 'Genotype', 'Age_group', 'ID', 'Organ_name', 'UMI_count', 'Gene_count', 'Immune_main_type', 'Immune_subtype', 'Immune_umap_1', 'Immune_umap_2', 'Doublet_scores', 'Sub_cell_type', 'Main_cell_type', 'Lineage', 'Sex', 'sample'
    var: 'gene_id', 'gene_type', 'gene_name'
    layers: 'raw_counts'

In [151]:
# separate the clusters
adata.obs['cluster_number'] = adata.obs['Sub_cell_type'].str.extract(r'-(\d+)-').astype(int)
adata.obs['tissue'] = adata.obs['Sub_cell_type'].str.split('-').str[-1]
adata.obs['cell_type'] = adata.obs['Sub_cell_type'].str.split('-').str[0].str.replace(' ', '_')
adata.obs['age'] = adata.obs['Age_group'].str.split('_').str[0].astype(int)
adata.obs['cluster_id'] = adata.obs['cell_type'].astype(str) + '_' + adata.obs['age'].astype(str) + 'm_' + adata.obs['Sex'].astype(str) + '_' + adata.obs['cluster_number'].astype(str)

In [152]:
# get lung and 3m to export and compare to TMS
lung_3m = adata[(adata.obs['tissue']== 'Lung') & (adata.obs['age']== 3)]

In [175]:
lung_3m

View of AnnData object with n_obs × n_vars = 128663 × 55416
    obs: 'Genotype', 'Age_group', 'ID', 'Organ_name', 'UMI_count', 'Gene_count', 'Immune_main_type', 'Immune_subtype', 'Immune_umap_1', 'Immune_umap_2', 'Doublet_scores', 'Sub_cell_type', 'Main_cell_type', 'Lineage', 'Sex', 'sample', 'cluster_number', 'tissue', 'cell_type', 'age', 'cluster_id'
    var: 'gene_id', 'gene_type', 'gene_name'
    layers: 'raw_counts'

In [180]:
# narrow down cell types
cell_type_list = list(lung_3m.obs['Immune_subtype'].unique())
narrowed_cell_type_list = ['Chil3+ alveolar macrophages', 
                           'Inflammatory monocytes',
                            'Patrolling monocytes',
                            'Col14a1+ Itga8+ macrophages',
                            'Proliferating macrophages',
                            'Lyve1+ Colec12+ macrophages',
                            'Type-2 conventional dendritic cells',
                            'Type-1 conventional dendritic cells',
                            'Migratory dendritic cells',
                            'Retnla+ Cd226+ macrophages',
                            'Cfd+ Scd1+ macrophages',
                            'Intestinal macrophages',
                            'Gpc6+ Hspg2+ macrophages',
                            'Kupffer cells',
                            'Mmp12+ Mmp19+ macrophages',
                            'Muscle-resident macrophages',
                            'Colq+ Gpm6b+ macrophages',
                            'Fcgr4+ Itgad+ macrophages']

In [ ]:
# filter object for new cell types 


Downsampling

In [153]:
n_cells_to_sample = 1200
np.random.seed(42)
random_indices = np.random.choice(adata.obs.index, size=n_cells_to_sample, replace=False)

lung_3m_small = adata[random_indices].copy()

In [154]:
lung_3m_small

AnnData object with n_obs × n_vars = 1200 × 55416
    obs: 'Genotype', 'Age_group', 'ID', 'Organ_name', 'UMI_count', 'Gene_count', 'Immune_main_type', 'Immune_subtype', 'Immune_umap_1', 'Immune_umap_2', 'Doublet_scores', 'Sub_cell_type', 'Main_cell_type', 'Lineage', 'Sex', 'sample', 'cluster_number', 'tissue', 'cell_type', 'age', 'cluster_id'
    var: 'gene_id', 'gene_type', 'gene_name'
    layers: 'raw_counts'

In [155]:
# set gene as index
lung_3m_small.var = lung_3m_small.var.set_index('gene_name')

/Users/helene/anaconda3/envs/pansci_env/lib/python3.10/site-packages/anndata/_core/anndata.py:750: UserWarning: 
AnnData expects .var.index to contain strings, but got values like:
    ['4933401J01Rik', 'Gm26206', 'Xkr4', 'Gm18956', 'Gm37180']

    Inferred to be: categorical

  value_idx = self._prep_dim_index(value.index, attr)


Calculate gene mean expression for each cell type

In [156]:
# Count the number of cells in each tissue group
lung_3m_small.obs.groupby('cluster_id').size()


cluster_id
Erythroblasts_16m_Female_0             1
Erythroblasts_6m_Male_0                1
Lymphoid_cells_16m_Male_7              1
Lymphoid_cells_3m_Female_4             1
Lymphoid_cells_6m_Male_4               1
                                      ..
Myeloid_cells_Neutrophils_6m_Male_0    1
Myeloid_cells_Neutrophils_6m_Male_1    1
Myeloid_cells_Neutrophils_6m_Male_2    1
Myeloid_cells_Neutrophils_6m_Male_7    1
Myeloid_cells_Neutrophils_6m_Male_9    1
Length: 441, dtype: int64

In [157]:
# Exclude cluster with only one cell
cluster_counts = lung_3m_small.obs['cluster_id'].value_counts()
single_cell_clusters = cluster_counts[cluster_counts == 1].index.tolist()
lung_3m_small = lung_3m_small[~lung_3m_small.obs['cluster_id'].isin(single_cell_clusters), :]
lung_3m_small.obs.groupby('cluster_id').size()
# there are super few cells per cluster??

cluster_id
Myeloid_cells_12m_Female_0               8
Myeloid_cells_12m_Female_1               3
Myeloid_cells_12m_Female_10              2
Myeloid_cells_12m_Female_2               4
Myeloid_cells_12m_Female_3               4
                                        ..
Myeloid_cells_Neutrophils_3m_Male_0      2
Myeloid_cells_Neutrophils_3m_Male_1      2
Myeloid_cells_Neutrophils_3m_Male_6      3
Myeloid_cells_Neutrophils_6m_Female_1    2
Myeloid_cells_Neutrophils_6m_Female_2    3
Length: 212, dtype: int64

In [158]:
lung_3m_small.layers["normalised"] = sc.pp.normalize_total(
    lung_3m_small, target_sum=1e4, inplace=False
)["X"]

/Users/helene/anaconda3/envs/pansci_env/lib/python3.10/site-packages/scanpy/preprocessing/_normalization.py:207: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/Users/helene/anaconda3/envs/pansci_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [159]:
lung_3m_small.layers["log_normalised"] = sc.pp.log1p(
    lung_3m_small.layers["normalised"], copy=True
)

In [160]:
# Calculate mean expression grouped by 'cluster_id'
sc.tl.rank_genes_groups(lung_3m_small, groupby='cluster_id', method='t-test', layer = 'log_normalised') 

/Users/helene/anaconda3/envs/pansci_env/lib/python3.10/site-packages/scanpy/tools/_rank_genes_groups.py:431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, "scores"] = scores[global_indices]
/Users/helene/anaconda3/envs/pansci_env/lib/python3.10/site-packages/scanpy/tools/_rank_genes_groups.py:434: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, "pvals"] = pvals[global_indices]
/Users/helene/anaconda3/envs/pansci_env/lib/python3.10/site-packages/scanpy/tools/_rank_genes_groups.py:444: P

In [161]:
print(lung_3m_small.uns['rank_genes_groups'].keys())

dict_keys(['params', 'names', 'scores', 'pvals', 'pvals_adj', 'logfoldchanges'])


In [162]:
ranked_genes = lung_3m_small.uns['rank_genes_groups']
genes = pd.DataFrame(ranked_genes['names']) 
log_means = pd.DataFrame(ranked_genes['logfoldchanges'])  
combined_df = pd.concat([genes, log_means], axis=1)



In [163]:

df_genes_long = pd.melt(
    genes.reset_index(), 
    id_vars=['index'], 
    var_name='cluster_id', 
    value_name='gene'
).rename(columns={'index': 'gene_id'})
df_genes_long

,gene_id,cluster_id,gene
0,0,Myeloid_cells_3m_Female_0,Abcg3
1,1,Myeloid_cells_3m_Female_0,Snx24
2,2,Myeloid_cells_3m_Female_0,Gm19951
3,3,Myeloid_cells_3m_Female_0,Hps3
4,4,Myeloid_cells_3m_Female_0,Pitpnc1
...,...,...,...
11748187,55411,Myeloid_cells_Neutrophils_23m_Male_4,Lyn
11748188,55412,Myeloid_cells_Neutrophils_23m_Male_4,Lrmda
11748189,55413,Myeloid_cells_Neutrophils_23m_Male_4,Gm19951
11748190,55414,Myeloid_cells_Neutrophils_23m_Male_4,Zeb2


In [164]:

df_expression_long = pd.melt(
    log_means.reset_index(), 
    id_vars=['index'], 
    var_name='cluster_id', 
    value_name='gmean'
).rename(columns={'index': 'gene_id'})
df_expression_long

,gene_id,cluster_id,gmean
0,0,Myeloid_cells_3m_Female_0,3.932500
1,1,Myeloid_cells_3m_Female_0,3.030553
2,2,Myeloid_cells_3m_Female_0,1.660643
3,3,Myeloid_cells_3m_Female_0,3.605592
4,4,Myeloid_cells_3m_Female_0,2.795742
...,...,...,...
11748187,55411,Myeloid_cells_Neutrophils_23m_Male_4,-32.085556
11748188,55412,Myeloid_cells_Neutrophils_23m_Male_4,-32.325768
11748189,55413,Myeloid_cells_Neutrophils_23m_Male_4,-32.271866
11748190,55414,Myeloid_cells_Neutrophils_23m_Male_4,-33.015305


In [165]:

df_combined = pd.merge(
    df_genes_long, 
    df_expression_long, 
    on=['gene_id', 'cluster_id']
)
df_combined

,gene_id,cluster_id,gene,gmean
0,0,Myeloid_cells_3m_Female_0,Abcg3,3.932500
1,1,Myeloid_cells_3m_Female_0,Snx24,3.030553
2,2,Myeloid_cells_3m_Female_0,Gm19951,1.660643
3,3,Myeloid_cells_3m_Female_0,Hps3,3.605592
4,4,Myeloid_cells_3m_Female_0,Pitpnc1,2.795742
...,...,...,...,...
11748187,55411,Myeloid_cells_Neutrophils_23m_Male_4,Lyn,-32.085556
11748188,55412,Myeloid_cells_Neutrophils_23m_Male_4,Lrmda,-32.325768
11748189,55413,Myeloid_cells_Neutrophils_23m_Male_4,Gm19951,-32.271866
11748190,55414,Myeloid_cells_Neutrophils_23m_Male_4,Zeb2,-33.015305


Sparsity

In [174]:
# check sparsity
from scipy.sparse import issparse
print("Is sparse: ", issparse(lung_3m_small.X)) # just sees wheather is it stored as dense or sparse

total_elements = lung_3m_small.X.shape[0] * lung_3m_small.X.shape[1] # total number of elements is calc by shape[0] and shape [1]
non_zero_elements = lung_3m_small.X.count_nonzero()
sparsity = (1 - (non_zero_elements / total_elements))*100
print(f"Sparsity: {sparsity}%")

Is sparse:  True
Sparsity: 99.18328992790343%


In [167]:
# save
df_combined.to_csv("../data/pansci_lung_3m_small.csv", index=False)